In [111]:
import numpy as np
import random

In [242]:
class Gridworld:

    def __init__(self, shape=(14,14), num_negative_tiles=0, starting_point=(1,1),num_walls=20):


        self.size = shape
        self.num_negative_tiles = num_negative_tiles
        self.current_state = np.array(starting_point)
        self.grid=None
        self.stochastic_transistions = np.zeros(shape=(shape[0]-1,shape[1]-1), dtype=np.float32)
        self.goal = np.array((shape[0]-2, shape[1]-2))
        self.acc_reward = 0 
        self.generate_world(num_walls)
        
    def generate_world(self, num_walls):
        valid_grid = False
        while not valid_grid:
            self.grid = self.generate_grid()
            unique_wall_indexes = False
            while not unique_wall_indexes:
                random_walls = np.random.randint(1,self.size[0]-1,size=(num_walls,2))
                random_walls_unique = np.unique(random_walls,axis=0)
                # check that the indeces neither describe the starting point nor the goal state or are doubled
                if len(random_walls) == len(random_walls_unique) and not tuple(self.goal) in random_walls.tolist() and not tuple(self.current_state) in random_walls.tolist():
                    unique_wall_indexes = True
            # apply for all unique indexes 
            for r in random_walls:
                self.grid[r[0],r[1]] = -100
            valid_grid = self.grid_is_valid()
    
    def generate_grid(self):
        grid = np.zeros(self.size, dtype=np.int64)
        grid[self.goal[0]][self.goal[1]] = 100
        grid[0,:] = -100
        grid[self.size[1]-1,:] = -100
        grid[:,0] = -100
        grid[:,self.size[1]-1] = -100
        return grid

          
    # depth first search to find out if the generated grid world is valid i.e there is a path from start to goal
    def grid_is_valid(self):
        visited = []
        visited = self.dfs(visited, self.current_state)     
        return np.any(np.all(self.goal == visited, axis=1))

    def dfs(self, visited, node):
        visited.append(node)
        neighbors = self.get_neighbors(node)
        for neighbor in neighbors:
            if not np.any(np.all(neighbor == np.array(visited), axis=1)):
            # neighbor is not in visited
                if self.grid[neighbor[0]][neighbor[1]] != -100:
                # neighbor is not a wall
                    self.dfs(visited, neighbor)
        return visited
            
    def get_neighbors(self, node):
        # edges in the same order as the step function, important !!!
        edges = [(0,1), (0,-1), (-1,0), (1,0)]
        neighbors = []
        for edge in edges:
            neighbor = (node[0] + edge[0], node[1] + edge[1])
            neighbors.append(np.array(neighbor))
        return neighbors
        
        
    # reset the actor to starting state
    def reset(self):
        self.current_state = (1,1)
        self.acc_reward = 0


    def step(self,action):
        '''
        Args:
        action(): 0: right, 1: left, 2, up, 3, down
        throws error if move is invalid due to wall
        '''
        
        # anders ugly mit if elif statements, switch erst ab python 3.10
        # right
        if action == 0:
            # get current state 
            y, x = self.current_state
            # check that current state is accessable
            new_y,new_x = y, x+1
            if self.grid[new_y,new_x] != -100:
                # update current state and collect rewward
                self.current_state = ((new_y, new_x))
                self.acc_reward += self.grid[new_y,new_x]
            else:
                raise ValueError('Could not move there due to wall.')


        # left step
        elif action == 1:
            # get current state 
            y, x = self.current_state
            # check that current state is accessable
            new_y,new_x = y, x-1
            if self.grid[new_y,new_x] != -100:
                # update current state and collect rewward
                self.current_state = ((new_y, new_x))
                self.acc_reward += self.grid[new_y,new_x]
            else:
                raise ValueError('Could not move there due to wall.')

        # upwards step
        elif action == 2:
            # get current state 
            y, x = self.current_state
            # check that current state is accessable
            new_y,new_x = y-1, x
            if self.grid[new_y,new_x] != -100:
                # update current state and collect rewward
                self.current_state = ((new_y, new_x))
                self.acc_reward += self.grid[new_y,new_x]
            else:
                raise ValueError('Could not move there due to wall.')
        
        # downwards step
        elif action == 3:
            # get current state 
            y, x = self.current_state
            # check that current state is accessable
            new_y,new_x = y+1, x
            if self.grid[new_y,new_x] != -100:
                # update current state and collect rewward
                self.current_state = ((new_y, new_x))
                self.acc_reward += self.grid[new_y,new_x]
            else:
                raise ValueError('Could not move there due to wall.')
        else:
            raise ValueError('Action index out of bounds. Actions-space = (0,1,2,3)')
    
        
    # print the gridworld as an array 
    def visualize(self):
        print(self.grid)







In [350]:
class SarsAgent:
    def __init__(self, grid_world, state, epsilon=0.9, alpha=0.5, gamma=0.95):
        self.learning_rate = alpha
        self.discount_factor = gamma
        self.epsilon = epsilon
        self.current_state = state
        self.grid_world = grid_world
        #self.size = tuple(np.append(np.subtract(grid_world.size, (2,2)), np.array(4)))
        self.size = tuple(np.append(np.array(grid_world.size), np.array(4)))
        self.q_table = np.zeros((self.size), dtype=np.float32)
        
    def get_reward(self,state):
        return self.grid_world.grid[state[0]][state[1]]
    
    def get_valid_actions(self, state):
        # get a list of all actions we can do for the next step
        actions = []
        grid = self.grid_world
        neighbors = grid.get_neighbors(state)
        for idx, neighbor in enumerate(neighbors):
            # if neighbor is not a wall, add action to move there as valid
            if grid.grid[neighbor[0]][neighbor[1]] != -100:
                actions.append(idx)
        return actions
      
    def choose_action(self, state):
        # choose the next action for the agent
        
        actions = self.get_valid_actions(state)
        #print(f'actions: {actions}')
        
        # choose random action
        if np.random.uniform(0,1) < self.epsilon:
            action = np.random.choice(actions)
        # choose highest q-value action
        else:
            valid_states = {}
            for i in actions:
                valid_states[i] = (self.q_table[state][i])
                
            #print(f'valid_states:{valid_states}')
            
            # argmax for dictionary
            action = max(valid_states, key=valid_states.get)
            
        #print(f'chosen action: {action}')
        return action
            
    def learn(self, n_steps):   
        state = self.grid_world.current_state
        next_action = None
        rewards = []
        for n in range(n_steps):
            # get rewards for calculating the q value
            reward = self.get_reward(state) * (self.discount_factor** n)
            rewards.append(reward)
            action = self.choose_action(state)
            if next_action == None:
            # save the first action to move 
                next_action = action
            
            # get the next state for n-step Sarsa
            state = self.do_fake_step(state, action)
        
        Q = self.q_table[self.current_state, next_action] 
        final_q = self.q_table[state, action] * (self.discount_factor** n_steps)
        Q = rewards + final_q - Q
        self.q_table[self.current_state, action] = Q
        #print(f'next_action: {next_action}')
        return next_action
        
        
    def do_fake_step(self, state, action):
        # get the next state if we move according to the given action
        grid = self.grid_world.grid     
        edges = [(0,1), (0,-1), (-1,0), (1,0)]
        state = (state[0] + edges[action][0], state[1] + edges[action][1])
        return state
        
        
    

In [355]:
grid = Gridworld()
#grid.step(0)
valid_states = grid.grid_is_valid()
grid.visualize()
#print(grid.stochastic_transistions)
agent = SarsAgent(grid, grid.current_state)


[[-100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100]
 [-100    0    0 -100 -100 -100    0    0 -100    0    0    0    0 -100]
 [-100    0    0    0    0    0    0    0    0    0 -100 -100    0 -100]
 [-100    0 -100    0    0    0    0 -100    0    0 -100 -100    0 -100]
 [-100    0    0    0    0    0    0    0    0    0    0    0    0 -100]
 [-100    0    0    0    0 -100    0    0    0    0    0    0    0 -100]
 [-100    0 -100    0    0    0    0 -100    0    0    0    0    0 -100]
 [-100    0    0    0    0    0    0    0    0    0    0    0    0 -100]
 [-100 -100    0    0    0    0    0    0    0    0    0    0 -100 -100]
 [-100    0 -100    0    0    0 -100    0    0    0    0    0    0 -100]
 [-100    0    0    0    0    0    0    0    0 -100    0    0 -100 -100]
 [-100    0    0    0    0    0    0    0    0    0    0    0    0 -100]
 [-100 -100    0    0    0    0    0    0    0    0    0    0  100 -100]
 [-100 -100 -100 -100 -100 -100 -100 -100 -100 -100

In [360]:
episodes = 10
for episode in range(episodes):
    grid.reset()
    #print(grid.current_state, tuple(grid.goal))
    while grid.current_state != tuple(grid.goal):
        action = agent.learn(1)
        grid.step(action)
        print(grid.current_state)
    print(agent.q_table)

(1, 2)
(2, 2)
(1, 2)
(1, 1)
(2, 1)
(3, 1)
(4, 1)
(3, 1)
(4, 1)
(4, 2)
(4, 3)
(3, 3)
(3, 4)
(3, 3)
(3, 4)
(2, 4)
(2, 3)
(2, 2)
(2, 1)
(3, 1)
(4, 1)
(3, 1)
(4, 1)
(4, 2)
(4, 1)
(5, 1)
(4, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(5, 1)
(6, 1)
(7, 1)
(6, 1)
(7, 1)
(6, 1)
(7, 1)
(6, 1)
(5, 1)
(6, 1)
(7, 1)
(7, 2)
(7, 1)
(7, 2)
(8, 2)
(7, 2)
(7, 1)
(7, 2)
(7, 1)
(6, 1)
(5, 1)
(4, 1)
(3, 1)
(2, 1)
(3, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(6, 1)
(5, 1)
(4, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(6, 1)
(5, 1)
(6, 1)
(7, 1)
(7, 2)
(8, 2)
(8, 3)
(8, 4)
(9, 4)
(9, 5)
(8, 5)
(8, 6)
(8, 5)
(9, 5)
(8, 5)
(8, 6)
(8, 7)
(9, 7)
(8, 7)
(8, 6)
(8, 5)
(8, 6)
(8, 5)
(8, 6)
(8, 5)
(8, 4)
(8, 5)
(8, 4)
(8, 3)
(7, 3)
(6, 3)
(7, 3)
(7, 4)
(8, 4)
(8, 3)
(7, 3)
(7, 4)
(7, 5)
(6, 5)
(6, 4)
(7, 4)
(8, 4)
(9, 4)
(9, 3)
(8, 3)
(7, 3)
(8, 3)
(8, 4)
(8, 5)
(8, 6)
(8, 7)
(8, 8)
(8, 9)
(9, 9)
(9, 10)
(9, 11)
(8, 11)
(8, 10)
(8, 11)
(8, 10)
(7, 10)
(8, 10)
(8, 11)
(9, 11)
(9, 10)
(8, 10)
(8, 9)
(8, 8)
(7, 8)
(6, 8)
(6, 9)
(

(2, 8)
(2, 9)
(1, 9)
(2, 9)
(3, 9)
(4, 9)
(3, 9)
(4, 9)
(5, 9)
(5, 10)
(6, 10)
(6, 9)
(5, 9)
(4, 9)
(4, 8)
(3, 8)
(2, 8)
(2, 9)
(3, 9)
(2, 9)
(3, 9)
(3, 8)
(4, 8)
(4, 9)
(4, 10)
(5, 10)
(6, 10)
(6, 9)
(6, 10)
(6, 11)
(5, 11)
(6, 11)
(6, 12)
(7, 12)
(7, 11)
(6, 11)
(5, 11)
(5, 10)
(4, 10)
(4, 11)
(5, 11)
(5, 10)
(5, 9)
(4, 9)
(4, 8)
(4, 7)
(4, 6)
(4, 5)
(4, 4)
(4, 3)
(4, 4)
(4, 5)
(3, 5)
(4, 5)
(4, 4)
(3, 4)
(3, 5)
(4, 5)
(4, 6)
(4, 7)
(5, 7)
(4, 7)
(4, 8)
(5, 8)
(5, 9)
(6, 9)
(6, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 11)
(5, 10)
(5, 11)
(4, 11)
(4, 10)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(5, 12)
(6, 12)
(5, 12)
(4, 12)
(4, 11)
(4, 10)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(3, 12)
(2, 12)
(3, 12)
(2, 12)
(1, 12)
(1, 11)
(1, 10)
(1, 9)
(1, 10)
(1, 9)
(1, 10)
(1, 11)
(1, 10)
(1, 11)
(1, 12)
(1, 11)
(1, 10)
(1, 9)
(2, 9)
(3, 9)
(2, 9)
(1, 9)
(2, 9)
(3, 9)
(3, 8)
(3, 9)
(3, 8)
(3, 9)
(2, 9)
(1, 9)
(2, 9)
(2, 8)
(2, 9)
(1, 9)
(1, 10)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 11)
(1, 10)
(1, 9)
(1, 10)
(1, 11)
(1, 12

(6, 3)
(7, 3)
(7, 2)
(7, 3)
(6, 3)
(5, 3)
(4, 3)
(4, 4)
(4, 5)
(3, 5)
(2, 5)
(3, 5)
(3, 6)
(4, 6)
(5, 6)
(4, 6)
(3, 6)
(2, 6)
(1, 6)
(1, 7)
(2, 7)
(2, 8)
(2, 9)
(1, 9)
(2, 9)
(2, 8)
(2, 9)
(2, 8)
(2, 7)
(2, 6)
(2, 7)
(2, 6)
(1, 6)
(1, 7)
(1, 6)
(1, 7)
(1, 6)
(1, 7)
(1, 6)
(2, 6)
(1, 6)
(2, 6)
(3, 6)
(2, 6)
(2, 5)
(2, 6)
(3, 6)
(2, 6)
(3, 6)
(3, 5)
(3, 6)
(4, 6)
(4, 5)
(4, 6)
(3, 6)
(4, 6)
(4, 7)
(4, 8)
(3, 8)
(2, 8)
(2, 7)
(2, 8)
(2, 9)
(3, 9)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(5, 12)
(6, 12)
(7, 12)
(7, 11)
(6, 11)
(6, 12)
(7, 12)
(6, 12)
(7, 12)
(6, 12)
(7, 12)
(6, 12)
(6, 11)
(6, 10)
(6, 11)
(5, 11)
(6, 11)
(6, 12)
(6, 11)
(7, 11)
(7, 12)
(7, 11)
(6, 11)
(5, 11)
(6, 11)
(6, 12)
(7, 12)
(7, 11)
(7, 10)
(7, 9)
(6, 9)
(6, 8)
(7, 8)
(6, 8)
(6, 9)
(7, 9)
(8, 9)
(8, 8)
(9, 8)
(10, 8)
(10, 7)
(11, 7)
(12, 7)
(12, 8)
(11, 8)
(11, 9)
(11, 8)
(12, 8)
(12, 9)
(12, 10)
(12, 11)
(11, 11)
(12, 11)
(12, 12)
[[[0.00e+00 0.00e+00 0.00e+00 0.00e+00]
  [0.00e+00 0.00e+00 0.00e+00 0.00e+00]
  [0.00e+00 0.

(7, 11)
(6, 11)
(5, 11)
(5, 10)
(6, 10)
(6, 11)
(6, 12)
(7, 12)
(6, 12)
(5, 12)
(5, 11)
(5, 12)
(4, 12)
(4, 11)
(4, 10)
(4, 9)
(4, 10)
(4, 11)
(4, 10)
(5, 10)
(5, 11)
(5, 10)
(6, 10)
(6, 11)
(6, 12)
(7, 12)
(6, 12)
(7, 12)
(6, 12)
(6, 11)
(7, 11)
(8, 11)
(9, 11)
(9, 12)
(9, 11)
(10, 11)
(11, 11)
(11, 12)
(12, 12)
[[[0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [3.e-44 3.e-44 3.e-44 3.e-44]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0

(11, 10)
(11, 11)
(11, 12)
(12, 12)
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 